In [ ]:
"""
This is an experimental notebook to walkthrough the process of getting relevant weather, crime, and
census data for the city of Philadelphia, PA.
"""

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import time

from datetime import datetime, timedelta


def fetch_crime_data(table: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Downloads Philadelphia crime data from OpenDataPhilly in the specified date range.

    Parameters:
    -----------
    table: str
        The name of the table in the OpenDataPhilly database.
    start_date: str
        The start date for the data in 'YYYY-MM-DD' format.
    end_date: str
        The end date for the data in 'YYYY-MM-DD' format.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the crime data with relevant columns.
    """
    print(f"Extracting crime data from {start_date} to {end_date}...")

    # Simple SQL query to get the data needed from the OpenDataPhilly database
    # Subsetting to the columns we need, some are left out because I think they are irrelevant

    # Variables Selected:
    # - dc_dist:         The police district where the incident occurred.
    # - psa:             The Police Service Area, a smaller geographic subdivision of a district.
    # - dispatch_date:   The date the call for service was dispatched (YYYY-MM-DD).
    # - dispatch_time:   The time the call for service was dispatched (HH:MI:SS).
    # - hour:            The hour of the day the call was dispatched (0-23).
    # - text_general_code: The text description for the type of incident (e.g., "Theft," "Assault").
    # - location_block:  The street address of the incident, anonymized to the block level.
    # - lat:             The latitude coordinate for the incident location (aliased from point_y).
    # - lon:             The longitude coordinate for the incident location (aliased from point_x).

    query = f"""
        SELECT dc_dist, psa, dispatch_date, dispatch_time, hour, text_general_code, location_block,
        point_y as lat, point_x as lon
        FROM {table}
        WHERE dispatch_date >= '{start_date}' AND dispatch_date <= '{end_date}'
    """

    # Connect to the OpenDataPhilly API and get the data
    response = requests.get(CARTO_URL, params={"q": query})
    response.raise_for_status()
    data = response.json().get("rows", [])
    crime_df = pd.DataFrame(data)

    return crime_df


def clean_crime_data(crime_df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the crime data to prepare for analysis. This involves steps like one-hot encoding,
    removing NAs, renaming columns, and creating new features.

    Parameters:
    ----------
    crime_df: pd.DataFrame
        The DataFrame containing the raw crime data.

    Returns:
    -------
    pd.DataFrame
        A cleaned DataFrame with relevant columns and features for analysis.
    """
    print("Cleaning crime data...")

    # Drop rows with missing latitude, longitude, or police service area (psa)
    crime_df = crime_df.dropna(subset=["lat", "lon", "psa"])

    # Rename columns for clarity
    crime_rename_map = {
        "dc_dist": "police_district",
        "psa": "police_service_area",
        "text_general_code": "crime_type",
        "location_block": "address_block",
    }
    crime_df = crime_df.rename(columns=crime_rename_map)

    # Convert police district and service area to dummy variables since they are categorial
    crime_df["police_district"] = crime_df["police_district"].astype(str)
    crime_df["police_service_area"] = crime_df["police_service_area"].astype(str)
    district_dummies = pd.get_dummies(crime_df["police_district"], prefix="district")
    psa_dummies = pd.get_dummies(crime_df["police_service_area"], prefix="psa")
    crime_df = crime_df.join(district_dummies)
    crime_df = crime_df.join(psa_dummies)
    crime_df = crime_df.drop(columns=["police_district", "police_service_area"])

    # Create dummy columns for every unique crime type
    dummies = pd.get_dummies(crime_df["crime_type"], prefix="crime")
    crime_df = crime_df.join(dummies)
    crime_df = crime_df.drop(columns=["crime_type"])

    # Convert relevant columns to datetime
    crime_df["dispatch_date_dt"] = pd.to_datetime(crime_df["dispatch_date"]).dt.date
    crime_df["dispatch_date"] = pd.to_datetime(crime_df["dispatch_date"])
    crime_df["dispatch_time"] = pd.to_datetime(crime_df["dispatch_time"])

    # Extract hour from dispatch_time; while the original dataframe had an hour column, it had missing
    # values, so I decided to do it myself
    crime_df["hour"] = crime_df["dispatch_time"].dt.hour

    # Define hour/month/day of week cylic features
    crime_df["hour_sin"] = np.sin(2 * np.pi * crime_df["hour"] / 24.0)
    crime_df["hour_cos"] = np.cos(2 * np.pi * crime_df["hour"] / 24.0)
    crime_df["month"] = crime_df["dispatch_date"].dt.month
    crime_df["month_sin"] = np.sin(2 * np.pi * crime_df["month"] / 12.0)
    crime_df["month_cos"] = np.cos(2 * np.pi * crime_df["month"] / 12.0)
    crime_df["day_of_week"] = crime_df["dispatch_date"].dt.dayofweek
    crime_df["day_of_week_sin"] = np.sin(2 * np.pi * crime_df["day_of_week"] / 7.0)
    crime_df["day_of_week_cos"] = np.cos(2 * np.pi * crime_df["day_of_week"] / 7.0)
    crime_df = crime_df.drop(columns=["hour", "month", "day_of_week"])

    # Print min and max dispatch date for sanity check to ensure the data is within the expected range
    min_date = crime_df["dispatch_date"].min()
    max_date = crime_df["dispatch_date"].max()
    print(f"Min dispatch_date: {min_date}")
    print(f"Max dispatch_date: {max_date}")

    return crime_df


def fetch_weather_data(
    station_id: str, start_date: str, end_date: str, token: str, max_retries: int = 5
) -> pd.DataFrame:
    """
    Downloads daily weather data from NOAA for a single station.
    This function is designed to be called for a limited date range (e.g., one year), to avoid
    overwhelming the API with a single large request.

    Parameters:
    ----------
    station_id: str
        The NOAA station ID for the weather station (e.g., "GHCND:USW00013739" for Philadelphia Intl Airport).
    start_date: str
        The start date for the data in 'YYYY-MM-DD' format.
    end_date: str
        The end date for the data in 'YYYY-MM-DD' format.
    token: str
        Your NOAA API token for authentication.
    max_retries: int
        The maximum number of retries for failed requests (default is 5).

    Returns:
    -------
    pd.DataFrame
        A DataFrame containing the weather data with relevant columns.
    """
    print(f"Fetching weather from {start_date} to {end_date}...")

    headers = {"token": token}
    all_results = []
    offset = 1

    # Get weather data from NOAA API
    # Variables Selected:
    # TMAX: Maximum Temperature
    # TMIN: Minimum Temperature
    # PRCP: Precipitation
    # AWND: Average Wind Speed
    # SNOW: Snowfall
    # SNWD: Snow Depth
    # NOTE: more can be added as per the documentation:
    # https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
    while True:
        params = {
            "datasetid": "GHCND",
            "stationid": station_id,
            "startdate": start_date,
            "enddate": end_date,
            "units": "standard",
            "limit": 1000,
            "offset": offset,
            "datatypeid": ["TMAX", "TMIN", "PRCP", "AWND", "SNOW", "SNWD"],
        }

        # Retry logic for temporary server errors
        response = None
        retries = 0
        while retries < max_retries:
            try:
                response = requests.get(
                    WEATHER_URL, headers=headers, params=params, timeout=20
                )
                if response.status_code in [500, 502, 503, 504]:
                    raise requests.exceptions.HTTPError(
                        f"{response.status_code} Server Error"
                    )
                response.raise_for_status()
                break
            except requests.exceptions.RequestException as e:
                wait_time = 2**retries
                print(f"Warning: Request failed ({e}). Retrying in {wait_time}s...")
                time.sleep(wait_time)
                retries += 1
        if not response or response.status_code != 200:
            print(f" Error: Failed to download data after {max_retries} retries.")
            return pd.DataFrame()

        results = response.json().get("results", [])
        if not results:
            break

        # Process the results and append to the list
        all_results.extend(results)
        offset += 1000
        time.sleep(0.2)

    # Format the results into a DataFrame and do some basic cleaning to help later on
    df = pd.DataFrame(all_results)
    if df.empty:
        return pd.DataFrame()
    df["date_dt"] = pd.to_datetime(df["date"]).dt.date
    weather_df = df.pivot_table(
        index="date_dt", columns="datatype", values="value"
    ).reset_index()

    return weather_df


def clean_weather_data(weather_df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the weather data to prepare for analysis. This involves renaming columns for now.

    Parameters:
    ----------
    weather_df: pd.DataFrame
        The DataFrame containing the raw weather data.

    Returns:
    -------
    pd.DataFrame
        A cleaned DataFrame with relevant columns and features for analysis.
    """
    # Rename columns for clarity
    weather_rename_map = {
        "AWND": "avg_wind_speed_mph",
        "PRCP": "precipitation_inches",
        "SNOW": "snowfall_inches",
        "SNWD": "snow_depth_inches",
        "TMAX": "max_temp_f",
        "TMIN": "min_temp_f",
    }
    weather_df = weather_df.rename(columns=weather_rename_map)

    return weather_df


def fetch_census_data(state_fips: str, county_fips: str, token: str) -> pd.DataFrame:
    """
    Downloads and processes ACS 5-Year data for all tracts in a county, including calculation of key
    demographic rates.

    Parameters:
    -----------
    state_fips: str
        The FIPS code for the state (e.g., "42" for Pennsylvania).
    county_fips: str
        The FIPS code for the county (e.g., "101" for Philadelphia County).
    token: str
        Your Census API token for authentication.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the census data with relevant columns and calculated rates.
    """
    print(
        f"Downloading census data for Philadelphia County (FIPS: {state_fips}{county_fips})..."
    )

    # Variables Selected:
    # NAME: Geographic Area Name
    # B01003_001E: Total Population
    # B19013_001E: Median Household Income
    # B17001_002E: Poverty Count (total people below poverty line)
    # B01002_001E: Median Age
    # B25002_001E: Total Housing Units
    # B25002_003E: Vacant Housing Units
    # B25003_001E: Total Occupied Housing Units
    # B25003_003E: Renter-Occupied Housing Units
    # NOTE: could get more variables, as listed here:
    # https://api.census.gov/data/2023/acs/acs5/variables.html
    variables = "NAME,B01003_001E,B19013_001E,B17001_002E,B01002_001E,B25002_001E,B25002_003E,B25003_001E,B25003_003E"
    params = {
        "get": variables,
        "for": "tract:*",
        "in": f"state:{state_fips} county:{county_fips}",
        "key": token,
    }

    # Make the request to the Census API and parse the response
    response = requests.get(CENSUS_API_URL, params=params)
    response.raise_for_status()
    data = response.json()
    census_df = pd.DataFrame(data[1:], columns=data[0])

    return census_df


def clean_census_data(census_df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the census data to prepare for analysis. This involves renaming columns for clarity and
    creating new columns.

    Parameters:
    ----------
    census_df: pd.DataFrame
        The DataFrame containing the raw census data.

    Returns:
    -------
    pd.DataFrame
        A cleaned DataFrame with relevant columns and features for analysis.
    """
    # Rename columns for clarity on what they represent
    census_df = census_df.rename(
        columns={
            "B01003_001E": "pop_total",
            "B19013_001E": "income_median",
            "B17001_002E": "poverty_total",
            "B01002_001E": "median_age",
            "B25002_001E": "total_housing_units",
            "B25002_003E": "vacant_housing_units",
            "B25003_001E": "total_occupied_units",
            "B25003_003E": "renter_occupied_units",
            "tract": "tract_fips_short",
        }
    )

    # Create the full FIPS code for joining
    census_df["tract_fips"] = (
        census_df["state"] + census_df["county"] + census_df["tract_fips_short"]
    )

    # Convert all relevant columns to numeric, handling errors
    numeric_cols = [
        "pop_total",
        "income_median",
        "poverty_total",
        "median_age",
        "total_housing_units",
        "vacant_housing_units",
        "total_occupied_units",
        "renter_occupied_units",
    ]
    for col in numeric_cols:
        census_df[col] = pd.to_numeric(census_df[col], errors="coerce")

    # Calculate new features which are useful for the clustering task
    # Replace zeros with NaN to prevent division errors
    census_df["pop_total"] = census_df["pop_total"].replace(0, np.nan)
    census_df["total_housing_units"] = census_df["total_housing_units"].replace(
        0, np.nan
    )
    census_df["total_occupied_units"] = census_df["total_occupied_units"].replace(
        0, np.nan
    )
    census_df["poverty_rate"] = census_df["poverty_total"] / census_df["pop_total"]
    census_df["vacancy_rate"] = (
        census_df["vacant_housing_units"] / census_df["total_housing_units"]
    )
    census_df["renter_occupancy_rate"] = (
        census_df["renter_occupied_units"] / census_df["total_occupied_units"]
    )

    # Replace any resulting NaN values (from division by zero) with 0
    rate_cols = ["poverty_rate", "vacancy_rate", "renter_occupancy_rate"]
    census_df[rate_cols] = census_df[rate_cols].fillna(0)

    print(f"Downloaded and processed census data for {len(census_df)} tracts.")

    # Return the final, clean set of columns
    final_columns = [
        "tract_fips",
        "pop_total",
        "income_median",
        "median_age",
        "poverty_rate",
        "vacancy_rate",
        "renter_occupancy_rate",
    ]

    return census_df[final_columns]


def get_census_tracts():
    """
    Fetches census tracts from OpenDataPhilly API and returns them as a GeoDataFrame.

    Returns:
    --------
    gpd.GeoDataFrame
        A GeoDataFrame containing the census tracts with their geometries.
    """
    print("Fetching census tracts from OpenDataPhilly API...")

    # Make the API call to get the GeoJSON data
    response = requests.get(CENSUS_SHAPE_URL)

    # This will raise an error if the request fails
    response.raise_for_status()

    # Load the GeoJSON response directly into a GeoDataFrame
    gdf_tracts = gpd.GeoDataFrame.from_features(response.json()["features"])

    # Set the coordinate reference system, which is standard for web data
    gdf_tracts = gdf_tracts.set_crs("EPSG:4326")

    return gdf_tracts


def merge_crime_census(
    crime_df: pd.DataFrame, census_tracts: gpd.GeoDataFrame
) -> pd.DataFrame:
    """
    Merges crime data with census data based on spatial join with census tracts.

    Parameters:
    ----------
    crime_df: pd.DataFrame
        The DataFrame containing the cleaned crime data.
    census_df: pd.DataFrame
        The DataFrame containing the cleaned census data.
    census_tracts: gpd.GeoDataFrame
        The GeoDataFrame containing the census tracts geometries.

    Returns:
    -------
    pd.DataFrame
        A merged DataFrame containing crime and census data.
    """
    # Convert the crime DataFrame to a GeoDataFrame
    print("Converting crime data to GeoDataFrame...")
    crime_gdf = gpd.GeoDataFrame(
        crime_df,
        geometry=gpd.points_from_xy(crime_df.lon, crime_df.lat),
        crs="EPSG:4326",  # Ensure CRS matches the census tract data
    )

    # Perform the spatial join
    print("Performing spatial join to map crimes to census tracts...")
    # Note: The 'predicate' argument was renamed to 'op' in recent geopandas versions
    try:
        # For newer GeoPandas versions
        final_crime_data = gpd.sjoin(crime_gdf, census_tracts, how="inner", op="within")
    except TypeError:
        # For older GeoPandas versions
        final_crime_data = gpd.sjoin(
            crime_gdf, census_tracts, how="inner", predicate="within"
        )

    print("\n Spatial join complete. Crime data now includes census tract info.")

    # Clean up the final DataFrame
    final_crime_data = final_crime_data.drop(
        columns=[
            "OBJECTID",
            "STATEFP10",
            "COUNTYFP10",
            "TRACTCE10",
            "NAME10",
            "NAMELSAD10",
            "MTFCC10",
            "FUNCSTAT10",
            "AWATER10",
            "INTPTLAT10",
            "INTPTLON10",
            "LOGRECNO",
            "index_right",
        ]
    )
    final_crime_map = {
        "GEOID10": "tract_id",
        "ALAND10": "land_area_sq_meters",
    }
    final_crime_data = final_crime_data.rename(columns=final_crime_map)

    return final_crime_data


def calculate_population_density(merged_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates population density for each census tract in the merged DataFrame.

    Parameters:
    ----------
    merged_df: pd.DataFrame
        The DataFrame containing the merged crime, weather, and census data.

    Returns:
    -------
    pd.DataFrame
        The DataFrame with an additional column for population density.
    """
    print("Calculating population density...")

    # Convert land_area_sq_meters to square kilometers, replacing 0 with NaN to avoid errors
    merged_df["area_sq_km"] = merged_df["land_area_sq_meters"] / SQ_METERS_PER_SQ_KM
    merged_df["area_sq_km"] = merged_df["area_sq_km"].replace(0, np.nan)

    # Calculate density and fill any missing results with 0
    merged_df["pop_density_sq_km"] = merged_df["pop_total"] / merged_df["area_sq_km"]
    merged_df["pop_density_sq_km"] = merged_df["pop_density_sq_km"].fillna(0)

    print("Population density calculated.")

    # Display the new column to verify the result
    print(
        merged_df[
            ["tract_id", "pop_total", "land_area_sq_meters", "pop_density_sq_km"]
        ].head()
    )

    # Drop the land_area_sq_meters column as it's no longer needed
    merged_df = merged_df.drop(columns=["land_area_sq_meters"])

    return merged_df

I setup some initial global parameters, such as relevant links to the data sources, and my 
tokens to connect with any APIs.

In particular, I use OpenDataPhilly to get the crime data (and I'll use it again for getting the 
census tract data for the Philadelphia region). I use NOAA to get weather data for each data from a
specific weather station, which is the Philadelphia International Airport. Finally, I use the 
official U.S. Census API to get census data.

*Note that ideally, I originally wanted to retrieve weather data more accurately, since there are 
multiple weather stations in Philadelphia. I would've just retrieved the data from said stations,
and then match each crime to the weather of the closest station. However, due to NOAA API request 
limitations for a free account, I simplified this step to get weather from one station to represent 
the entire region. Since the region is relatively small (compared to if I did this for say, the
whole state of Pennsylvania), this should still be relatively accurate.*

In [ ]:
# Initialize tokens:
# NOTE: Actual key is hidden for security reasons, but you can use the following links to get your
# own key to connect with the APIs
# NOAA: https://www.ncdc.noaa.gov/cdo-web/token
# Census: https://api.census.gov/data/key_signup.html
NOAA_TOKEN = ""
CENSUS_TOKEN = ""

# Define crime data sources
CARTO_URL = "https://phl.carto.com/api/v2/sql"
CRIME_TABLE_NAME = "incidents_part1_part2"

# Define weather station source for NOAA API
# TODO: Incorperate weather station data from multiple stations
WEATHER_STATION_ID = "GHCND:USW00013739"  # Philadelphia Intl Airport
WEATHER_URL = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

# Define census data sources
FCC_CENSUS_URL = "https://geo.fcc.gov/api/census/block/find"
# Using 2019-2023 ACS 5-Year estimates
CENSUS_API_URL = "https://api.census.gov/data/2023/acs/acs5"  #
# Philadelphia County FIPS code is 101, and the state FIPS code for Pennsylvania is 42.
# The weather station ID for Philadelphia Intl Airport is GHCND:USW0001373
PHILLY_COUNTY_FIPS = "101"
PA_STATE_FIPS = "42"

# The URL for the census tract shapes, which is used to map crimes to census tracts
CENSUS_SHAPE_URL = "https://hub.arcgis.com/api/v3/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1"

# Define date range to collect date (using a 3-year rolling window)
# Contains data up to yesterday, since you cannot get all the data for current date
END_DATE = datetime.now() - timedelta(days=1)
START_DATE = END_DATE - timedelta(days=3 * 365)
START_STR = START_DATE.strftime("%Y-%m-%d")
END_STR = END_DATE.strftime("%Y-%m-%d")

# Conversion factor from square meters to square kilometers
SQ_METERS_PER_SQ_KM = 1_000_000

Now, I get the crime data.

In [16]:
# Get and clean the crime data
crime_df = fetch_crime_data(CRIME_TABLE_NAME, START_STR, END_STR)
crime_df = clean_crime_data(crime_df)

# Print out some basic information about the crime data as sanity check
print(crime_df.head())
print(crime_df.shape)
print(crime_df.info())

Extracting crime data from 2022-07-08 to 2025-07-07...
Cleaning crime data...


C:\Users\kevin\AppData\Local\Temp\ipykernel_20132\445612047.py:110: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crime_df['dispatch_time'] = pd.to_datetime(crime_df['dispatch_time'])


Min dispatch_date: 2022-07-08 00:00:00
Max dispatch_date: 2025-07-07 00:00:00
  dispatch_date       dispatch_time                    address_block  \
0    2025-05-02 2025-07-08 23:44:00              1100 BLOCK S 4TH ST   
1    2025-04-23 2025-07-08 18:18:00  300 BLOCK MARTIN LUTHER KING DR   
2    2025-04-23 2025-07-08 18:32:00             1700 BLOCK N 32ND ST   
3    2025-04-24 2025-07-08 00:47:00          1300 BLOCK W Venango St   
4    2025-02-26 2025-07-08 20:16:00              400 BLOCK N 35TH ST   

         lat        lon  district_01  district_02  district_03  district_05  \
0  39.934248 -75.150833        False        False         True        False   
1  39.962990 -75.178868        False        False        False        False   
2  39.983588 -75.186199        False        False        False        False   
3  40.007425 -75.149843        False        False        False        False   
4  39.961642 -75.192723        False        False        False        False   

   district_06

Next, I get the weather data. Since I have to split up my API calls to avoid errors, I have to use
some extra code to get all the data in chunks, and then combine them.

In [17]:
# Get weather data
print("Processing weather data year-by-year...")

# Initialize an empty list to hold the DataFrames for each year
weather_frames = []
# Calculate the years within your 3-year date range
start_year = START_DATE.year
end_year = END_DATE.year

# Loop through each year in the range and fetch the weather data
for year in range(start_year, end_year + 1):
    # Define the start and end for this specific year's API call
    year_start_str = f"{year}-01-01"
    year_end_str = f"{year}-12-31"
    print(f"-> Downloading weather for year {year}...")
    df_year = fetch_weather_data(
        WEATHER_STATION_ID, year_start_str, year_end_str, NOAA_TOKEN
    )

    if not df_year.empty:
        weather_frames.append(df_year)

# If I successfully downloaded data for any year, combine them into a single DataFrame
if weather_frames:
    # Combine the data from all successful yearly calls
    weather_df = pd.concat(weather_frames, ignore_index=True)
    # Filter the combined data to the precise 3-year rolling window
    weather_df["date_dt_obj"] = pd.to_datetime(weather_df["date_dt"])
    weather_df = weather_df[
        (weather_df["date_dt_obj"] >= pd.to_datetime(START_STR))
        & (weather_df["date_dt_obj"] <= pd.to_datetime(END_STR))
    ].drop(columns=["date_dt_obj"])
    print(f"\n Successfully combined weather data for {len(weather_df)} days.")
else:
    print("\n No weather data could be downloaded.")
    weather_df = pd.DataFrame()

weather_df = clean_weather_data(weather_df)

# Print out some basic information about the weather data as sanity check
print(weather_df.head())
print(weather_df.shape)
print(weather_df.info())

Processing weather data year-by-year...
-> Downloading weather for year 2022...
Fetching weather from 2022-01-01 to 2022-12-31...
-> Downloading weather for year 2023...
Fetching weather from 2023-01-01 to 2023-12-31...
-> Downloading weather for year 2024...
Fetching weather from 2024-01-01 to 2024-12-31...
-> Downloading weather for year 2025...
Fetching weather from 2025-01-01 to 2025-12-31...

 Successfully combined weather data for 1094 days.
datatype     date_dt  avg_wind_speed_mph  precipitation_inches  \
188       2022-07-08                 6.0                  0.00   
189       2022-07-09                 3.6                  0.02   
190       2022-07-10                 5.8                  0.00   
191       2022-07-11                 6.7                  0.00   
192       2022-07-12                12.8                  0.00   

datatype  snowfall_inches  snow_depth_inches  max_temp_f  min_temp_f  
188                   0.0                0.0        88.0        68.0  
189      

Now, I get the census data.

In [18]:
# Get and clean census data
census_df = fetch_census_data(PA_STATE_FIPS, PHILLY_COUNTY_FIPS, CENSUS_TOKEN)
census_df = clean_census_data(census_df)

# Print out some basic information about the census data as sanity check
print(census_df.head())
print(census_df.shape)
print(census_df.info())

Downloaded and processed census data for 408 tracts.
    tract_fips  pop_total  income_median  median_age  poverty_rate  \
0  42101000101     1996.0         108438        32.1      0.016032   
1  42101000102     3025.0         108203        33.0      0.058182   
2  42101000200     3259.0          97256        38.9      0.235655   
3  42101000300     4236.0         102330        34.8      0.064684   
4  42101000401     2857.0          89663        33.7      0.178509   

   vacancy_rate  renter_occupancy_rate  
0      0.067500               0.746649  
1      0.100044               0.622725  
2      0.074881               0.535017  
3      0.064710               0.808845  
4      0.117572               0.683983  
(408, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tract_fips             408 non-null    object 
 1   pop_to

At this point, the most important parts of the data are stored. However, to properly map the census
data to the crime data, I need to determine which census tract each crime is located in.

To do this, I need to get the geojson file from OpenDataPhilly, which is a file that effectively 
outlines each census tract in the city of Philadelphia.

In [19]:
# Get Census Tract geojson from OpenDataPhilly, to help map crime locations to census tracts
# TODO: This currently retrieves tracts from 2010, since that's the most recent option available.
# Of course, this is not ideal, and should be switched to 2020 when available.
gdf_tracts = get_census_tracts()

# Print out some basic information about the census tracts as sanity check
print(gdf_tracts.head())

Fetching census tracts from OpenDataPhilly API...
                                            geometry  OBJECTID STATEFP10  \
0  POLYGON ((-75.22927 39.96054, -75.22865 39.960...         1        42   
1  POLYGON ((-75.23536 39.96852, -75.23545 39.969...         2        42   
2  POLYGON ((-75.24343 39.9623, -75.24339 39.9622...         3        42   
3  POLYGON ((-75.17341 39.97779, -75.17386 39.977...         4        42   
4  POLYGON ((-75.17313 39.97776, -75.17321 39.977...         5        42   

  COUNTYFP10 TRACTCE10      GEOID10 NAME10        NAMELSAD10 MTFCC10  \
0        101    009400  42101009400     94   Census Tract 94   G5020   
1        101    009500  42101009500     95   Census Tract 95   G5020   
2        101    009600  42101009600     96   Census Tract 96   G5020   
3        101    013800  42101013800    138  Census Tract 138   G5020   
4        101    013900  42101013900    139  Census Tract 139   G5020   

  FUNCSTAT10  ALAND10  AWATER10   INTPTLAT10    INTPTLON10 L

This gives a lot of information for each census tract. Though, it has a lot of columns that is not
neccessary. Regardless, my next step is to convert my crime data into a geo-dataframe, allowing me
to merge it with the census tract with a spatial join. I also clean up the data to remove a lot of 
the unnecessary columns.

In [20]:
# Map crime locations to census tracts using spatial joins
final_crime_data = merge_crime_census(crime_df, gdf_tracts)

# Display the result
print(final_crime_data.head())

Converting crime data to GeoDataFrame...
Performing spatial join to map crimes to census tracts...

 Spatial join complete. Crime data now includes census tract info.
  dispatch_date       dispatch_time                    address_block  \
0    2025-05-02 2025-07-08 23:44:00              1100 BLOCK S 4TH ST   
1    2025-04-23 2025-07-08 18:18:00  300 BLOCK MARTIN LUTHER KING DR   
2    2025-04-23 2025-07-08 18:32:00             1700 BLOCK N 32ND ST   
3    2025-04-24 2025-07-08 00:47:00          1300 BLOCK W Venango St   
4    2025-02-26 2025-07-08 20:16:00              400 BLOCK N 35TH ST   

         lat        lon  district_01  district_02  district_03  district_05  \
0  39.934248 -75.150833        False        False         True        False   
1  39.962990 -75.178868        False        False        False        False   
2  39.983588 -75.186199        False        False        False        False   
3  40.007425 -75.149843        False        False        False        False   
4  39

Now, all three datasets are ready to be merged together into one big dataset!

In [21]:
print("Merging crime and weather data...")
crime_and_weather_df = pd.merge(
    final_crime_data,
    weather_df,
    left_on="dispatch_date_dt",
    right_on="date_dt",
    how="left",
)
print("Merging with census data...")
merged_df = pd.merge(
    crime_and_weather_df,
    census_df,
    left_on="tract_id",
    right_on="tract_fips",
    how="left",
)

# Clean up by dropping redundant key columns
merged_df = merged_df.drop(columns=["date_dt", "tract_fips"], errors="ignore")
merged_df = merged_df.dropna(subset=["tract_id", "pop_total"])

# As always, print out some basic information about the final merged dataset for sanity check
print("\n Merge complete!")
print("Columns in the final dataset:")
print(merged_df.columns)
print("\nSample of the fully merged data:")
print(merged_df.head())
print(merged_df.info())

Merging crime and weather data...
Merging with census data...

 Merge complete!
Columns in the final dataset:
Index(['dispatch_date', 'dispatch_time', 'address_block', 'lat', 'lon',
       'district_01', 'district_02', 'district_03', 'district_05',
       'district_06', 'district_07', 'district_08', 'district_09',
       'district_12', 'district_14', 'district_15', 'district_16',
       'district_17', 'district_18', 'district_19', 'district_22',
       'district_24', 'district_25', 'district_26', 'district_35',
       'district_39', 'district_77', 'psa_1', 'psa_2', 'psa_3', 'psa_4',
       'psa_5', 'psa_A', 'crime_Aggravated Assault Firearm',
       'crime_Aggravated Assault No Firearm', 'crime_All Other Offenses',
       'crime_Arson', 'crime_Burglary Non-Residential',
       'crime_Burglary Residential', 'crime_DRIVING UNDER THE INFLUENCE',
       'crime_Disorderly Conduct', 'crime_Embezzlement',
       'crime_Forgery and Counterfeiting', 'crime_Fraud',
       'crime_Gambling Violati

As one final step, I go ahead and add a population density column. This can be important for the
clustering, since crime certainly might appear more often/less often based on how many people are 
in a local census tract.

In [22]:
merged_df = calculate_population_density(merged_df)

Calculating population density...
Population density calculated.
      tract_id  pop_total  land_area_sq_meters  pop_density_sq_km
0  42101002500     4306.0               398697       10800.181592
2  42101014900     4140.0               453221        9134.616445
3  42101020300     2807.0               508176        5523.676836
4  42101009000     7525.0               436319       17246.555846
5  42101028600     7170.0               717845        9988.228657


Now, I just remove any rows with missing rows, since we cannot have missing values for clustering.
At this point, these are likely due to user error and make up the very small minority of our 
dataset, so I don't go through the trouble to try and impute them.

In [23]:
# Drop any remaining missing values; these are just due to user error and are rare cases, and are
# not worth the trouble of trying to fill in
final_merged_df = merged_df.dropna()
assert final_merged_df.isna().sum().sum() == 0, "Error: Missing values were found!"

# Print out the final merged DataFrame information
final_merged_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 413660 entries, 0 to 463369
Data columns (total 88 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   dispatch_date                                  413660 non-null  datetime64[ns]
 1   dispatch_time                                  413660 non-null  datetime64[ns]
 2   address_block                                  413660 non-null  object        
 3   lat                                            413660 non-null  float64       
 4   lon                                            413660 non-null  float64       
 5   district_01                                    413660 non-null  bool          
 6   district_02                                    413660 non-null  bool          
 7   district_03                                    413660 non-null  bool          
 8   district_05                              

For further experimentation, I save the data as a static pickle file. This will be used in the next
experimental notebook.

In [26]:
filepath = f"data/merged_data_{START_STR}_to_{END_STR}.pkl"
final_merged_df.to_pickle(filepath)
print(f"\nFinal merged data saved to {filepath}.")


Final merged data saved to data/merged_data_2022-07-08_to_2025-07-07.pkl.


This is just a code cell to remind me to ensure my token strings are blank before commiting, since
it is meant to be private and commiting it would be a security concern!

In [25]:
BOLD = "\033[1m"
RED = "\033[31m"
RESET = "\033[0m"

# Print the bold red text
print(
    f"{BOLD}{RED}MAKE SURE TO SET NOAA AND CENSUS TOKEN TO BE EMPTY BEFORE COMMITING!{RESET}"
)

MAKE SURE TO SET NOAA AND CENSUS TOKEN TO BE EMPTY BEFORE COMMITING!
